In [1]:
pip install openai-whisper --quiet

In [2]:
pip install google-cloud-aiplatform --quiet

In [3]:
pip install -q -U google-genai --quiet

In [4]:
!pip install streamlit google-generativeai pydub pyngrok --quiet

In [5]:
pip install gTTS

In [6]:
import whisper

# Load the Whisper model
model = whisper.load_model("base")

# Transcribe the audio file
def transcribe_audio(file_path):
    result = model.transcribe(file_path)
    return result['text']

In [7]:
transcript = transcribe_audio('/content/Walking_thoughts_3.m4a')
print(transcript)

 So, right, so early on in my career, I spent a lot of time kind of working on fissions and road maps and ideas and you know some of them ended up not happening at all. I remember us throwing away like large projects or canceling them late on. It still happens to me today. It still happens to people on my team today. Yeah, priorities change and I think that is yeah, market change was lots of reasons for it, but I remember being so frustrated because I think I came straight out of school with this idea that you're given an assignment and then you do it. And there is no like takes these backsees on an assignment. It's like here's the content we need from you. Now produce it and then you're graded on it. That's so wildly different than entering in the workforce and it became really difficult to try and like ratify this idea that I just had to work and produce things with none of it. My mattress in the day, none of it might be used, none of it, or any of it might not go anywhere. And it fe

In [8]:
from google import generativeai as genai
genai.configure(api_key="YOUR_API_KEY")

def summarize_text(text):
    model = genai.GenerativeModel(model_name="gemini-2.0-flash")
    response = model.generate_content(
        contents=[f"Please summarize the following text:\n\n{text}"]
    )
    return response.text
summary = summarize_text(transcript)
print(summary)

The speaker describes a frustrating early career experience where projects were frequently canceled, leading to a feeling of wasted effort. They contrast this with the structured academic environment they were used to. However, joining a new team with a "write to think" culture shifted their perspective. They realized that even work that gets scrapped is valuable because it contributes to skill development and learning. The "write to think" approach encourages frequent, early writing with the understanding that not everything will be a final product, promoting iteration and learning. They now advocate for this approach, emphasizing that there is no "throwaway work" in this model, as it all contributes to improvement.



In [9]:
summary

'The speaker describes a frustrating early career experience where projects were frequently canceled, leading to a feeling of wasted effort. They contrast this with the structured academic environment they were used to. However, joining a new team with a "write to think" culture shifted their perspective. They realized that even work that gets scrapped is valuable because it contributes to skill development and learning. The "write to think" approach encourages frequent, early writing with the understanding that not everything will be a final product, promoting iteration and learning. They now advocate for this approach, emphasizing that there is no "throwaway work" in this model, as it all contributes to improvement.\n'

In [10]:
from gtts import gTTS
import os

def text_to_speech(text):
    tts = gTTS(text)
    tts.save("summary.mp3")

In [11]:
text_to_speech(summary)

In [12]:
%%writefile app.py
import streamlit as st
import google.generativeai as genai
import os
import tempfile
import whisper
from gtts import gTTS
# Configure Gemini API key
genai.configure(api_key="YOUR_API_KEY")

whisper_model = whisper.load_model("base")
def transcribe_audio(file_path):
    result = whisper_model.transcribe(file_path)
    return result['text']

def summarize_text(text):
    try:
        model = genai.GenerativeModel("gemini-2.0-flash")
        response = model.generate_content([f"Please summarize the following text:\n\n{text}"])
        return response.text
    except Exception as e:
        return f"An error occurred during summarization: {e}"
def text_to_speech(text):
    tts = gTTS(text)
    tts.save("summary.mp3")
# Streamlit app
def main():
    summary = ''
    st.title("Audio Summarizer using Gemini")


    uploaded_audio = st.file_uploader("Upload an audio file (MP3 or WAV)", type=["mp3", "wav",'m4a'])

    if uploaded_audio is not None:
        with tempfile.NamedTemporaryFile(delete=False, suffix=".wav") as tmp:
            tmp.write(uploaded_audio.read())
            tmp_path = tmp.name

        st.audio(uploaded_audio)


        st.subheader("Transcribed Text")
        transcript = transcribe_audio(tmp_path)
        st.text_area("Transcript", transcript, height=300)


        if st.button("Summarize Text"):
            summary = summarize_text(transcript)
            st.subheader("Summary")
            st.write(summary)
            text_to_speech(summary)
            with open("summary.txt", "w") as text_file:
              text_file.write(summary)

            # Provide the download button for summary text
            st.download_button(
                label="Download Summary Text",
                data=open("summary.txt", "r").read(),
                file_name="summary.txt",
                mime="text/plain"
            )
            # Provide the download button for summary audio
            with open("summary.mp3", "rb") as audio_file:
                st.download_button(
                    label="Download Summary Audio",
                    data=audio_file,
                    file_name="summary.mp3",
                    mime="audio/mp3"
                )

if __name__ == "__main__":
    main()


Overwriting app.py


In [13]:
!pip install pyngrok
from pyngrok import ngrok
ngrok.set_auth_token("ngrok_creid")


In [14]:
!streamlit run app.py &>/content/logs.txt &

In [15]:
public_url = ngrok.connect(8501)
print(f"🔗 Your Streamlit app is live at: {public_url}")

🔗 Your Streamlit app is live at: NgrokTunnel: "https://6311-34-16-175-185.ngrok-free.app" -> "http://localhost:8501"
